In [6]:
import json
import pdfplumber
import spacy
import pandas as pd
import numpy as np
import os
import unidecode
from spacy.matcher import PhraseMatcher
from spacy import displacy
from progress_bar.progress_bar import printProgressBar

class School:
    def __init__(self, file_source):
        file = open(file_source, "r")
        file = json.load(file)
        temp_list = []
        for facultad in file:
            temp_list.append(facultad['escuela'])
        #print(facultad['escuela'])
        self.escuelas = [item for sublist in temp_list for item in sublist] # make the list flat
        #have the escuelas with accents in the correct form here
        self.escuelas_accent = self.escuelas
        
        print(self.escuelas)
        self.i = 0
        self.j = 0
        self.k = 0
        
    def create_dictionary(self, schools):
        myDict = dict((e,i) for i,e in enumerate(schools))
        return myDict

    def unaccent_list(self, accent_list):
        unaccented_schools = []
        for sch in accent_list:
            unaccented_schools.append(unidecode.unidecode(sch).lower())
        return unaccented_schools
    
    def set_school_to_unaccent(self):
        self.escuelas = self.unaccent_list(self.escuelas)
        
    def create_dicts(self):
        #create the dicts only when schools are unaccented
        self.escuelas_unaccent_dict = self.create_dictionary(self.escuelas)
        self.escuelas_accent_dict = self.create_dictionary(self.escuelas_accent)
        
    def set_schools_accents(self, row, l):
        self.k+= 1
        printProgressBar(self.k, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        
        index = self.escuelas_unaccent_dict.get(row.lower())
        key_list = list(self.escuelas_accent_dict.keys())
        val_list = list(self.escuelas_accent_dict.values())
        try:
            position = val_list.index(index)
            return key_list[position]
        except:
            return None
        #return the value of the position, example in dict '{..., Escuela de enfermería: 37, ...}' it will return 
        #'Escuela de enfermería'
    
    def clean_spaces_text(self, text):
        new_text = " ".join(text.split())
        return(new_text)
    
    def set_nlp(self, model):
        self.nlp_model = spacy.load(model)
        
    def set_matcher(self):
        self.matcher = PhraseMatcher(self.nlp_model.vocab, attr="LOWER")
        patterns = [self.nlp_model(name) for name in self.escuelas]
        self.matcher.add("ESC", patterns)
        
    def check_file(self, file_source, l):
        self.i+=1
        printProgressBar(self.i, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        pages_10 = []
        pages_10_l = []
        school_name_of_file = ""
        with pdfplumber.open(file_source) as pdf:
            for i in range(0,10):
                pages_10.append(self.clean_spaces_text(pdf.pages[i].extract_text()))
            for i in reversed(range(len(pdf.pages)-10,len(pdf.pages))):
                pages_10_l.append(self.clean_spaces_text(pdf.pages[i].extract_text()))
        
        #first 10 pages
        pages_10_u = self.unaccent_list(pages_10)
        for page in pages_10_u:
            doc = self.nlp_model(page)
            if len(self.matcher(doc)) >=1:
                for match_id, start, end in self.matcher(doc):
                    return(doc[start:end]) #returns at the first instance
        #last 10 pages
        pages_10_l = self.unaccent_list(pages_10_l)
        for page in pages_10_l:
            doc = self.nlp_model(page)
            if len(self.matcher(doc)) >=1:
                for match_id, start, end in self.matcher(doc):
                    return(doc[start:end]) #returns at the first instance
        return "No school"
    def create_training_set(self, file_source, l):
        self.j+=1
        printProgressBar(self.j, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
        pages_10 = []
        pages_10_l = []
        school_name_of_file = ""
        with pdfplumber.open(file_source) as pdf:
            for i in range(0,10):
                pages_10.append(self.clean_spaces_text(pdf.pages[i].extract_text()))
            for i in reversed(range(len(pdf.pages)-10,len(pdf.pages))):
                pages_10_l.append(self.clean_spaces_text(pdf.pages[i].extract_text()))
        
        #first 10 pages
        pages_10_u = self.unaccent_list(pages_10)
        for page in pages_10_u:
            doc = self.nlp_model(page)
            if len(self.matcher(doc)) >=1:
                for match_id, start, end in self.matcher(doc):
                    aux_training = [page, {"entities": (start, end, 'ESC')}]
                    return aux_training #returns at the first instance
        #last 10 pages
        pages_10_l = self.unaccent_list(pages_10_l)
        for page in pages_10_l:
            doc = self.nlp_model(page)
            if len(self.matcher(doc)) >=1:
                for match_id, start, end in self.matcher(doc):
                    aux_training = [page, {"entities": (start, end, 'ESC')}]
                    return aux_training #returns at the first instance
        return "No school"
    def print_training_set(self):
        print(self.training_set)
        

In [7]:
schools = School("data/escuelas.json")

['Escuela de Ingeniería Civil', 'Escuela de Ingeniería Eléctrica', 'Escuela de Geología, Minas y Geofísica', 'Escuela de Ingeniería Química', 'Escuela de Ingeniería de Petroleo', 'Escuela de Ingeniería Mecanica', 'Escuela de Ingeniería Metalúrgica y Ciencias de los Materiales', 'Escuela de Agronomía', 'Escuela de Arquitectura', 'Escuela de Biología', 'Escuela de Computación', 'Escuela de Física', 'Escuela de Geoquímica', 'Escuela de Matemática', 'Escuela de Química', 'Escuela de Administración y Contaduría', 'Escuela de Antropología', 'Escuela de Estadística y Ciencias Actuariales', 'Escuela de Economía', 'Escuela de Estudios Internacionales', 'Escuela de Sociología', 'Escuela de Trabajo Social', 'Escuela de Derecho', 'Escuela de Estudios Políticos y Administrativo', 'Escuela de Ciencias Veterinarias', 'Escuela de Farmacia', 'Escuela de Artes', 'Escuela de Bibliotecología y Archivología', 'Escuela de Comunicación Social', 'Escuela de Educación', 'Escuela de Filosofía', 'Escuela de Geog

In [8]:
#delete accents
schools.set_school_to_unaccent()

In [9]:
#load model set matcher for schools
schools.set_nlp('es_core_news_sm')
schools.set_matcher()

In [10]:
import pandas as pd
import numpy as np
import os

## get dataframe with text only thesis
csv_source = "data/url_thesis_200_resumen_with_scan.csv"
df = pd.read_csv(csv_source)
df = df[df['isScan']==False]
df = df.sort_values('isScan', ascending=False)

In [11]:
#size of data frame
l = len(df.index)
l

189

In [12]:
#make this fucntion a vectorize so it can run in a data frame
check_vec = np.vectorize(schools.check_file)

In [ ]:
schools.create_training_set("../thesis_pdf/1.pdf", 1)

In [13]:
#create a list of values with the school column, the l is the total ammount of thesis to check
values_c = check_vec("../"+df["path"],l)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [14]:
#append to dataframe the column of the matching school and create a csv
df['school_simple'] = values_c.tolist()
df.to_csv("./data/thesis_200_resumen_with_school.csv", index=False)

In [15]:
## get dataframe with only schools tag thesis
csv_source = "./data/thesis_200_resumen_with_school.csv"
df = pd.read_csv(csv_source)
df = df[df['isScan']==False]
df = df.sort_values('isScan', ascending=False)

In [16]:
#size of data frame
l = len(df.index)
l

189

In [17]:
schools.create_dicts()

In [18]:
schools.escuelas_unaccent_dict

{'escuela de ingenieria civil': 0,
 'escuela de ingenieria electrica': 1,
 'escuela de geologia, minas y geofisica': 2,
 'escuela de ingenieria quimica': 3,
 'escuela de ingenieria de petroleo': 4,
 'escuela de ingenieria mecanica': 5,
 'escuela de ingenieria metalurgica y ciencias de los materiales': 6,
 'escuela de agronomia': 7,
 'escuela de arquitectura': 8,
 'escuela de biologia': 9,
 'escuela de computacion': 10,
 'escuela de fisica': 11,
 'escuela de geoquimica': 12,
 'escuela de matematica': 13,
 'escuela de quimica': 14,
 'escuela de administracion y contaduria': 15,
 'escuela de antropologia': 16,
 'escuela de estadistica y ciencias actuariales': 17,
 'escuela de economia': 18,
 'escuela de estudios internacionales': 19,
 'escuela de sociologia': 20,
 'escuela de trabajo social': 21,
 'escuela de derecho': 22,
 'escuela de estudios politicos y administrativo': 23,
 'escuela de ciencias veterinarias': 24,
 'escuela de farmacia': 25,
 'escuela de artes': 26,
 'escuela de biblio

In [19]:
# make the correct data frame first then vectorize
check_vec_accents = np.vectorize(schools.set_schools_accents)
schools.create_dicts()

In [20]:
df['school_simple']

0                escuela de ingenieria electrica
130               escuela de ingenieria mecanica
121                escuela de ingenieria quimica
122    escuela de bibliotecologia y archivologia
123               escuela de ingenieria mecanica
                         ...                    
65                         escuela de enfermeria
66                         escuela de enfermeria
67                         escuela de enfermeria
68                         escuela de enfermeria
188                        escuela de enfermeria
Name: school_simple, Length: 189, dtype: object

In [21]:
schools.create_dicts()

In [22]:
values_accent = check_vec_accents(df['school_simple'],l)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [23]:
values_accent

array(['Escuela de Ingeniería Eléctrica',
       'Escuela de Ingeniería Mecanica', 'Escuela de Ingeniería Química',
       'Escuela de Bibliotecología y Archivología',
       'Escuela de Ingeniería Mecanica',
       'Escuela de Geología, Minas y Geofísica',
       'Escuela de Bibliotecología y Archivología',
       'Escuela de Geología, Minas y Geofísica',
       'Escuela de Ingeniería Mecanica', 'Escuela de Ingeniería Química',
       'Escuela de Ingeniería Química', 'None',
       'Escuela de Geología, Minas y Geofísica',
       'Escuela de Ingeniería Química', 'None', 'None',
       'Escuela de Ingeniería Química', 'Escuela de Ingeniería Química',
       'Escuela de Ingeniería Química', 'Escuela de Ingeniería Química',
       'None', 'Escuela de Ingeniería Química', 'None',
       'Escuela de Ingeniería de Petroleo', 'Escuela de Economía',
       'Escuela de Ingeniería Química', 'Escuela de Ingeniería Mecanica',
       'Escuela de Geología, Minas y Geofísica',
       'Escuela de Ing

In [24]:
#append to dataframe the column of the matching school and create a csv with the correct name
df['school_complex'] = values_accent.tolist()
df.to_csv("./data/thesis_200_with_resumen_school_complex.csv", index=False)